In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = "./"
res_list = os.listdir(path)
res_list = [res for res in res_list if res.endswith(".txt")]
res_list.sort()
print(res_list)

['ARID3A_K562_ARID3A_(sc-8821)_Stanford-0.txt', 'CTCFL_K562_CTCFL_(SC-98982)_HudsonAlpha-0.txt', 'ELK1_GM12878_ELK1_(1277-1)_Stanford-0.txt', 'FOXA1_HepG2_FOXA1_(SC-101058)_HudsonAlpha-0.txt', 'GABPA_GM12878_GABP_HudsonAlpha-0.txt', 'MYC_H1-hESC_c-Myc_Stanford-0.txt', 'REST_GM12878_NRSF_HudsonAlpha-0.txt', 'SP1_GM12878_SP1_HudsonAlpha-0.txt', 'USF1_HepG2_USF-1_HudsonAlpha-0.txt', 'ZBTB7A_HepG2_ZBTB7A_(SC-34508)_HudsonAlpha-0.txt']


In [3]:
with pd.ExcelWriter('hyperparameter-performance.xlsx') as writer:
    for res_idx in range(len(res_list)):
        auc_over_epoch = []
        validation_auc = []
        test__auc = []

        hyp = []
        name = res_list[res_idx].split('_')[0]
        print(name)

        file = open(res_list[res_idx], 'r')
        while True:
            line = file.readline()
            if not line: break
            elif 'Train AUC over epochs' in line:
                auc_over_epoch.append(line)
            elif 'Best Trainig(validation) AUC' in line:
                validation_auc.append(line)
            elif 'AUC Test' in line:
                test__auc.append(line)
            elif 'pool' in line:
                hyp.append(line)
        file.close()

        if(len(auc_over_epoch) == 144):
        # some experiments are not done yet

            # plot AUC-epoch graphs
            for id in range(len(auc_over_epoch)): # id : 1~144

                data = auc_over_epoch[id].split(',')[:-1]
                train = []
                valid = []

                for i in range(len(data)):
                    train.append(float(data[i].split(':')[-2:][0]))
                    valid.append(float(data[i].split(':')[-2:][1]))

                isExist = os.path.exists('./' + name + '/')
                if not isExist:
                    os.makedirs('./' + name + '/')
                
                plt.plot(train, label='Train')
                plt.plot(valid, label='Validation')
                plt.legend()
                plt.xlabel('epochs')
                plt.ylabel('AUC')
                # plt.axis([0, 300, 0.8, 1.0])
                plt.savefig('./' + name + '/' + str(id+1) + '.png', dpi=300)
                plt.close()

            df = pd.DataFrame(columns=['hyperparameters', 'AUC_Train', 'AUC_Test'])

            for idx in range(len(validation_auc)):
                train = float(validation_auc[idx].split('\t')[0].split(':')[-1].split('at')[0])
                test = float(test__auc[idx].split('\t')[0].split(':')[-1])
                hyperparameters = hyp[idx].split('\n')[0]
                
                df.loc[idx] = [hyperparameters, train, test]
            
            df.to_excel(writer, sheet_name=name, index=False)

        else:
            print(name, 'is not done yet')

ARID3A
CTCFL
ELK1
FOXA1
GABPA
MYC
REST
SP1
USF1
ZBTB7A
